In [1]:
from os import listdir
from os.path import isfile, join
import re
import csv
import gensim
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText
import logging
import os
import pandas as pd 
import sys
import sklearn.metrics
import sklearn
import matplotlib.pyplot as plt #For graphics
import seaborn #Makes the graphics look nicer
import random
import math
import warnings
import utils
from utils import *
from utils2 import *
import scipy.stats
from dataDictionariesLexicons import *
from testData import *
from polesConstructs import *
from scipy import stats
random.seed(1)

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
# warnings.filterwarnings('ignore')

sys.path.insert(0, '../') # necessary in order to import word2vecTraining where the epoch saver class, which is part of the model, is located

In [2]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
# modelFilePath = os.path.join(modelsPath, 'GoogleNews-vectors-negative300.bin')
# model2 = KeyedVectors.load_word2vec_format(modelFilePath,binary=True) 
modelFilePath = os.path.join(modelsPath, 'GoogleNews-vectors-negative300LowerCase.bin')
model1 = KeyedVectors.load(modelFilePath) 

In [3]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'glove.6B.300d.txt.word2vec.bin')
model2 = KeyedVectors.load_word2vec_format(modelFilePath,binary=True) 

In [4]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'glove.twitter.27B.200d.txt.word2vec.bin')
model3 = KeyedVectors.load(modelFilePath) 

In [5]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'glove.42B.300d.txt.word2vec.bin')
model4 = KeyedVectors.load(modelFilePath) 

In [6]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'glove.840B.300d.txt.word2vec.bin')
model5 = KeyedVectors.load(modelFilePath) 

In [7]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'fastext-common-crawl-300d-2M-subword.bin')
model6 = KeyedVectors.load(modelFilePath)

In [8]:
modelsPath = os.path.normpath(r'D:\data\wordVectors') 
modelFilePath = os.path.join(modelsPath, 'fastext-wiki-news-300d-1M-subword-Original.bin')
model7 = KeyedVectors.load(modelFilePath)

In [9]:
# models = [model1,model2,model3,model4]
# models = [model1,model2]
models = [model1,model2,model3,model4,model5,model6,model7]
# models = [model0]

In [10]:
#change default lexicons
# lexicons = [harvardGeneralInquirer3623, vaderLexicon7062,NRCEmotionLexicon5555,opinionLexicon6786,afinnLexicon2477]
lexicons = LexiconsEnsembl
dataDict = dataDictLoadLexicons(dataDict,lexicons)
# WEAT1 harvardGeneralInquirer3623

In [11]:
dataSetIndexes = [
# '0-1', #percentage females per occupation
# '0-2', #country's GDP per capita
# '0-3', #car brands and prices
# '0-4', #voting behavior by ethnic/racial group
# '0-5', #voting behavior by profession
    
'1-1', #gender
'1-2',
'1-3',
'1-4',
'1-5',
    
'2-1', #Race/ethnicity
'2-2',
'2-3',
'2-4',    
  
'3-1', #Sexual orientation  
    
'4-1', #religiosity    
'4-2', #religion Islam and Christianity
    
'5-1', #age 
'6-1', #socioeconomic status
'7-1', #physical beauty
    
'8-1', #Personal ideology
'8-2', #Party affiliation and political parties
'8-3', #US presidents
'8-4', #Ideologies abstract
'8-5', #influential conservatives and liberals
]

In [12]:
rows = len(dataSetIndexes)
columns = len(LexiconsEnsembl)
outputMatrix = np.zeros((rows,columns))
outputMatrixPvalues = np.zeros((rows,columns))

## Dimensions against lexicons

In [13]:
for rowNumber, dataSetIndex in enumerate(dataSetIndexes):
    print(dataDict[dataSetIndex]['name'])
    lexiconsCorrelations=[]
    lexiconsPvalues=[]
    for RealDataTemp in dataDict[dataSetIndex]['RealDataLexicons']:
    
#         print("model", i+1)
        correlations = []
        pvalues = []
        for i,model in enumerate(models):    
            constructPole1 = dataDict[dataSetIndex]['constructPole1']
            constructPole2 = dataDict[dataSetIndex]['constructPole2']
        
            if i>=1: #Glove and fastext vocabulary uses - instead of _ So we need to alter Construct poles and real data
                constructPole1,constructPole2,RealDataTemp=replaceUnderscoreForDash(constructPole1,constructPole2,RealDataTemp)
            if i in [1,2,3]: # Lowercase poles for models With only lowercase vocabularies
                constructPole1 = list(set(list(map(str.lower,constructPole1))))
                constructPole2 = list(set(list(map(str.lower,constructPole2))))
            constructPole1,constructPole2 = constructsFilter(model,constructPole1,constructPole2,printFlag=False)# Filter out terms not in model vocabulary
            
            RealData = realDataFilter(model,RealDataTemp,0)#Filter out terms not in model vocabulary (Last parameter Flag prints out OOV words)
            Axis = dimensionN(model, constructPole1,constructPole2)  
            AxisName='Axis'
            try:
                dataFrame = makeDF(model, RealData.keys(),Axis,AxisName)
            except:
                continue
            dataFrame['RealValues']=RealData.values()
            spearmanCorr,pearsonCorr=calculateCorrelations(dataFrame,printFlag=False)
#             correlations.append(pearsonCorr[0])
#             pvalues.append(pearsonCorr[1])
            correlations.append(spearmanCorr[0])
            pvalues.append(spearmanCorr[1])
            
#         print("correlations for model", i+1)
#         print(*correlations)
#         print("mean: ", np.array(correlations).mean())
        lexiconsCorrelations.append(np.array(correlations).mean())
        chi2=-2*np.sum(np.log(np.array(pvalues))) #fisher's method to combine p-values
        combinedPvalue=1-stats.chi2.cdf(24.1864,2*len(pvalues))#degrees of freedom 2* number of tests being combined
        lexiconsPvalues.append(combinedPvalue)
#         print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
    print(*lexiconsCorrelations)
    print(np.array(lexiconsCorrelations).mean())
    outputMatrix[rowNumber]=lexiconsCorrelations
    #Print closest neighbors to axis Poles
#     print([w[0] for w in model.wv.similar_by_vector(-Axis,restrict_vocab=50000,topn=50)])
#     print([w[0] for w in model.wv.similar_by_vector(Axis,restrict_vocab=50000,topn=50)])

    print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

gender general
0.18672585146117335 0.025144509697644313 0.10733853659449702 0.13983571994620464 0.13539638425218026 0.10962872299680372 0.17047658727418225 0.18006920990664588 0.21654418588770982 0.19679675359518498 0.28197712382342066 0.13603283752516265 0.23038860654884058 0.2800173457664394 0.12444485206650473 0.21632195907730756 0.11520960386596822
0.16778522295799234
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender family roles
0.0905881397139843 0.12691047808023626 0.08800832559625985 0.05935358916854107 0.09449279944757179 0.08103387544175747 0.05921159140007641 0.10123178215026894 0.2097299511560123 0.2620266598318182 0.02950090947028927 0.0656123009113774 0.13140418636718695 0.14604650398939406 -0.007311849464293895 0.06115144995968611 0.07344625909796972
0.09837864425400801
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
gender and young age
0.1465019625272627 0.1387897740003832 0.07541693663893718 0.09988539369505826 0.10014785201017092 0.04754023358793994 0.0839

C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in log
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in log
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in log
C:\Users\drozado\miniconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in log


0.5432620747229784 0.8004665567525667 0.4938537773705804 0.5485984899967754 0.5638060423342104 0.5543415326627738 0.6623549059473403 0.7136333416694312 0.6844220596301742 0.6507102827067665 0.6713954840991488 0.6283827454507984 0.4587588632307166 0.23262292107321017 0.5099445113643776 0.3577257242145442 0.6533672691725945
0.5722145048469994
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
personal ideology
0.2988982854433154 0.3846911995474245 0.24292942338545917 0.24810234700593328 0.29656082952947393 0.28657898994221187 0.4581612067019152 0.5592191350671747 0.3358694632827412 0.48555431224962736 0.472233857107399 0.32448821414161744 0.27992075483865925 0.31727345160062664 0.2676454952395714 0.20024895007287744 0.4083549394297758
0.3451018149756355
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
party affiliation and political parties
0.10961304047620517 0.19501844135574528 0.08311694245487064 0.07969906883001945 0.11200089232752189 0.06720295674062939 0.16134707694808953 0.22057

In [14]:
for r in outputMatrix:
    print(*r)

0.18672585146117335 0.025144509697644313 0.10733853659449702 0.13983571994620464 0.13539638425218026 0.10962872299680372 0.17047658727418225 0.18006920990664588 0.21654418588770982 0.19679675359518498 0.28197712382342066 0.13603283752516265 0.23038860654884058 0.2800173457664394 0.12444485206650473 0.21632195907730756 0.11520960386596822
0.0905881397139843 0.12691047808023626 0.08800832559625985 0.05935358916854107 0.09449279944757179 0.08103387544175747 0.05921159140007641 0.10123178215026894 0.2097299511560123 0.2620266598318182 0.02950090947028927 0.0656123009113774 0.13140418636718695 0.14604650398939406 -0.007311849464293895 0.06115144995968611 0.07344625909796972
0.1465019625272627 0.1387897740003832 0.07541693663893718 0.09988539369505826 0.10014785201017092 0.04754023358793994 0.08394896530444486 0.09808011005595361 0.1598935193359104 0.17264251250808155 0.030432136332963128 0.10922762018424612 0.10852737643296809 0.20683214894985166 0.019816759356146466 0.1406397665119089 0.01

In [15]:
for r in outputMatrixPvalues:
    print(*r)

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

In [16]:
outputMatrix.mean(axis=1)

array([ 0.16778522,  0.09837864,  0.10328228,  0.21945214,  0.00662644,
       -0.09368303, -0.20371984,  0.00559444, -0.04762695,  0.04041276,
       -0.10287519, -0.21295432,  0.16130446,  0.25840478,  0.5722145 ,
        0.34510181,  0.15511038,  0.23992946,  0.19048565,  0.29788603])

## Correlation of results between different lexicons

In [17]:
#Calculating correlation of measurements across models
df = pd.DataFrame(outputMatrix)
print(df.iloc[:, 0].corr(df.iloc[:, 6]))
df.corr()

0.922292231750072


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1.000000,0.893804,0.972206,0.982016,0.977115,0.954208,0.922292,0.849095,0.931213,0.949740,0.812896,0.977385,0.928464,0.681417,0.680712,0.880698,0.952798
1,0.893804,1.000000,0.952579,0.874260,0.927299,0.948109,0.788539,0.715108,0.927314,0.895388,0.735228,0.884155,0.798045,0.522089,0.494160,0.734803,0.922899
2,0.972206,0.952579,1.000000,0.945369,0.970847,0.970019,0.877291,0.802569,0.952410,0.942489,0.764599,0.949597,0.869711,0.612042,0.577662,0.823796,0.949380
3,0.982016,0.874260,0.945369,1.000000,0.982595,0.963715,0.961286,0.907059,0.908134,0.937068,0.883566,0.994367,0.934328,0.628730,0.783910,0.869839,0.967968
4,0.977115,0.927299,0.970847,0.982595,1.000000,0.988810,0.938846,0.881235,0.929256,0.960402,0.868438,0.990774,0.900949,0.591906,0.728350,0.853521,0.982974
5,0.954208,0.948109,0.970019,0.963715,0.988810,1.000000,0.919919,0.868830,0.935965,0.939540,0.865560,0.970807,0.879839,0.549420,0.693278,0.834225,0.980166
6,0.922292,0.788539,0.877291,0.961286,0.938846,0.919919,1.000000,0.980217,0.831549,0.905806,0.921360,0.966770,0.887093,0.586457,0.842426,0.795136,0.946259
7,0.849095,0.715108,0.802569,0.907059,0.881235,0.868830,0.980217,1.000000,0.738572,0.864831,0.930114,0.914839,0.852554,0.573761,0.881725,0.726736,0.902778
8,0.931213,0.927314,0.952410,0.908134,0.929256,0.935965,0.831549,0.738572,1.000000,0.886989,0.717910,0.906855,0.797067,0.492813,0.506042,0.759131,0.901871
9,0.949740,0.895388,0.942489,0.937068,0.960402,0.939540,0.905806,0.864831,0.886989,1.000000,0.821810,0.947991,0.914746,0.697847,0.687251,0.761539,0.950606


In [18]:
correlationMatrix = np.array(df.corr())
allPairwiseCorrelations = correlationMatrix[np.triu_indices(17,k=1)] #If you want to extract the values that are above the diagonal (or below) then use the k argument.
allPairwiseCorrelations.mean()

0.8401511116001668